In [2]:
!pip install polars


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl

_file_location = './yellow_tripdata_2015-01.csv'

lf = (
    pl.scan_csv(_file_location)
    .select('tpep_pickup_datetime', 'VendorID', 'passenger_count', 'trip_distance')
    .with_columns(pl.col('tpep_pickup_datetime').str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S"))
    .filter(pl.col('passenger_count') < 3)
    .group_by('VendorID')
    .agg(
        pl.len().alias('trips'),
        pl.col('passenger_count').sum().alias('passengers'),
        pl.col('trip_distance').sum().alias('distance_driven')
    )
)

print(lf.explain())



AGGREGATE
	[len().alias("trips"), col("passenger_count").sum().alias("passengers"), col("trip_distance").sum().alias("distance_driven")] BY [col("VendorID")] FROM
  Csv SCAN [./yellow_tripdata_2015-01.csv]
  PROJECT 3/19 COLUMNS
  SELECTION: [(col("passenger_count")) < (3)]


In [3]:
%time result = lf.collect(streaming=True)

CPU times: user 2.99 s, sys: 1.97 s, total: 4.96 s
Wall time: 2.21 s
